In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/python-code-instruction-dataset")

print("Path to dataset files:", path)

In [ ]:
from datasets import load_dataset

# 1. Load CSV
dataset = load_dataset("csv", data_files="your_file.csv")["train"]  # single file is loaded as 'train' split

# 2. Select only 2 columns
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["text", "author"]])

# 3. Apply transformation -> make a single column
def transform(example):
    # Example transformation: concatenate text and author
    return {"input": example["text"] + " - " + example["author"]}

dataset = dataset.map(transform)

# Drop the original columns
dataset = dataset.remove_columns(["text", "author"])

# 4. Split into train/val/test
splits = dataset.train_test_split(test_size=0.2, seed=42)  # 80/20
test_valid = splits["test"].train_test_split(test_size=0.5, seed=42)  # split 20% into 10/10
final_dataset = {
    "train": splits["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
}

print(final_dataset)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    inputs =  tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Using device: cpu


Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir='/home/kamal/Desktop/Prodigy/PRODIGY_GA_01/results',
    eval_strategy='epoch',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=500,
    save_total_limit=2,  
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/home/kamal/Desktop/Prodigy/PRODIGY_GA_01/logs'
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)
trainer.train()


/home/kamal/Desktop/Prodigy/PRODIGY_GA_01/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
asasasas

In [ ]:
import pandas as pd
df=pd.read_csv('../train.csv')

In [ ]:
dataset.data['test']

------

In [ ]:
with open("../gemini/gemini_21.txt", "r") as f:
    kk = f.readlines()
print(kk)

In [ ]:
import os
path = '/home/kamal/Desktop/Prodigy/PRODIGY_GA_01/gemini/'
file_len_dict = {}
for i in os.listdir(path):
    file_len_dict[i] = len("\n".join(open(path+i).readlines()))

In [ ]:
file_len_dict

In [ ]:
len("".join(kk))

In [ ]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

text = "I want to check how many tokens I am giving to GPT-2."

# Encode into tokens
tokens = tokenizer.encode(text)

print("Tokens:", tokens)
print("Number of tokens:", len(tokens))

In [ ]:
len(tokenizer.get_vocab())

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless=new")

driver = webdriver.Chrome(options=chrome_options)
driver.get(r"https://www.flipkart.com/mens-tshirts/pr?sid=clo,ash,ank,edy&fm=neo%2Fmerchandising&iid=M_624b7d2f-334d-4126-866e-04f303731af5_1_X1NCR146KC29_MC.91BAWC5K7TBY&otracker=hp_rich_navigation_2_1.navigationCard.RICH_NAVIGATION_Fashion~Men%27s%2BTop%2BWear~Men%27s%2BT-Shirts_91BAWC5K7TBY&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L2_view-all&cid=91BAWC5K7TBY")

In [ ]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
from langchain_text_splitters import HTMLSemanticPreservingSplitter
splitter = HTMLSemanticPreservingSplitter([("h1", "Header 1"), ("h2", "Header 2")],
                                          max_chunk_size=500,chunk_overlap=50,
                                          elements_to_preserve=["table", "ul"]
                                          )
docs = splitter.split_text(str(soup.body))

In [ ]:
len(docs)

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("""
You are an expert in web data extraction.
You will be provided with a website URL. Your task is to extract only the requested information from the website.

**Guidelines:**

1. Extract only the information that actually exists on the website.

* If a piece of information is missing, do not output that, only output the existing information.
2. For items such as **privacy policy, return policy, terms & conditions, etc.** that appear as links, return only the URL in this format:

```json
{{ "the_key": {{ "url": "link" }} }}
```
3. Do not generate, guess, or add any information that is not present.
4. Do not include any explanations, reasoning, or text outside of the required JSON structure. Only output the JSON object.
5. Ensure the JSON is valid, properly formatted, and strictly follows the schema below.

**Output Format (strict JSON):**

{{"Hero Products":
    {{"product-name":"Name of the product",
    "product-url":"url of the product",
    "image-url":"url of the image",
    "price":"price of the product"}}
"Privacy Policy":
    {{"1":"first policy",
    "2":"the second policy",
    ...}}
"Return, Refund Policies":
    {{"1":"the policy",
    "2":"the second policy",
    ...}}
"Social Handles":
    {{"facebook":"url",
    "instagram":"url",
    ...}}
"Contact Details":
    {{"phone number":"the number",
    "email":"email_id",
    ...}}
"About the Brand":"about the brand details",
"Important links":
    {{"Order tracking":"the url",
    "Contact Us":"contact us url",
    "Blogs":["url1","url2"]
    ...}}
}}
Here is a chunk of html code of the website:
{html_code}
""")

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
hf = HuggingFacePipeline(pipeline=pipe)
chain2 = prompt_template | hf

In [ ]:
model.generate(['hi'])

In [ ]:
prompt_template.invoke("Hello")

In [ ]:
hf.invoke('who are you')

In [ ]:
chain2.invoke({'html_code':docs[1]})

In [ ]:
gpt_reponse = [get_response(i) for i in docs]

In [ ]:
the_list = [len(tokenizer.encode(prompt_template.invoke({"html_code": i.page_content}).text)) for i in docs]

In [ ]:
[i for i in the_list if i>=1000]

In [ ]:
import numpy as np
np.mean(the_list)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv('/home/kamal/.env')
import os
gemini_api = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=gemini_api)
chain = prompt_template | llm

In [ ]:
result = chain.invoke({'html_code':docs[0].page_content})

In [ ]:
result.content

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
llm2 = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm2.invoke("Hello, how are you?")

In [ ]:
chain1 = prompt_template | llm2

In [ ]:
kkkk=r"""i will provide you latex code for my resume and you will always provide me the full latex code of optimised resume. do not change the latex resume design, style or blueprint. Highlight matched keyworks with bold letters (\textbf). Do not give preamble or explanation or any otherthings except the full latex resume, unless i asked you to. Here is my resume ------------------- \section{SUMMARY} \begin{onecolentry} IBM-certified Data Scientist with a proven track record in designing and deploying data-driven solutions. Leveraged advanced statistical analysis, machine learning (including LSTM and NLP), and automation techniques to drive a 35\% increase in user engagement and reduce manual processes by up to 40\% during internships and open-source projects. Seeking to bring analytical rigor and innovative problem-solving to a forward-thinking organization. \end{onecolentry} \section{SKILLS} \begin{onecolentry} \begin{highlights} \item \textbf{Programming:} Python (NumPy, Pandas, Matplotlib, Seaborn, Plotly, Scikit-Learn, TensorFlow, Keras, PyTorch, Streamlit, OpenCV) \item \textbf{Data Analysis:} Statistical Analysis, Data Visualization, Data Preprocessing, Exploratory Data Analysis (EDA) \item \textbf{Machine Learning:} Supervised \& Unsupervised Learning, LSTM Networks, Natural Language Processing (NLP), Classification (Decision Trees, SVM, Random Forests), Clustering (K-means, Hierarchical) \item \textbf{Deep Learning:} Artificial Neural Networks (ANN), Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), Image Classification, Object Detection \item \textbf{Agentic Tools:} LangChain, LangGraph \item \textbf{Tools:} PostgrSQL, MySQL, MongoDB, Power BI, Git \& GitHub, Microsoft Excel, PowerBI, Docker \item \textbf{Soft Skills:} Communication, Problem Solving, Critical Thinking, Team Collaboration, Time Management \end{highlights} \end{onecolentry} \section{EXPERIENCE} \begin{twocolentry}{ \textit{Aug 2025 - Present} } \textbf{Generative AI Intern}, Prodigy InfoTech | Remote \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Designed and deployed a Markov Chain-based text generator, experimenting with token-level prediction to generate diverse and coherent text outputs, enhancing text automation capabilities. \item Fine-tuned GPT-2 on custom datasets to retrieve relevant information from web-scraped HTML pages, improving context relevance and knowledge grounding by 30\%. \item Applied \textbf{Pix2Pix GANs} for style transfer between images, successfully transforming visual aesthetics while maintaining structural integrity, showcasing deep learning creativity in real-world applications. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{twocolentry}{ \textit{Aug 2024 - Dec 2024} } \textbf{Open Source Contributor}, Resume Matcher | \href{https://resume-matcher-c.streamlit.app/}{\underline{WebSite}} | \href{https://github.com/KamalMahanna/Resume-Matcher.git}{\underline{GitHub}} \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Automated the creation of missing directories to resolve critical project initialization issues. \item Fixed Docker-compose initialization failuare by implementing automated directory creation, eliminating deployment failures. \item Integrated flake8 with git commit hooks to enforce code quality standards across the project. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{twocolentry}{ \textit{Jul 2023 - Oct 2023} } \textbf{Data Science Intern}, Industry Academia Community | \href{https://iac-chatbot.streamlit.app/}{\underline{WebSite}} | \href{https://github.com/KamalMahanna/IAC-Chatbot}{\underline{GitHub}} \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Developed an AI-powered chatbot using Google’s Dialogflow, increasing user engagement by 35\% among a community of over 5,000 members. \item Leveraged natural language processing (NLP) techniques to enhance response accuracy by 25\%. \item Reduced support response times by 20\% through efficient design and implementation. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{twocolentry}{ \textit{Jul 2023 - Aug 2023} } \textbf{Artificial Intelligence Intern}, Bharat Intern | \href{https://stock-price-kkm.streamlit.app/}{\underline{WebSite}} | \href{https://github.com/kamalmahanna/stock-price-prediction}{\underline{GitHub}} \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Engineered a stock price prediction model using LSTM networks, achieving RMSE values of 0.00045 (univariate) and 0.00016 (multivariate). \item Performed exploratory data analysis (EDA) to identify key patterns, enhancing forecasting accuracy by 15\%. \item Applied advanced statistical techniques on historical trading data to improve ROI for trading portfolios. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{twocolentry}{ \textit{Oct 2022 - Apr 2023} } \textbf{Data Science Intern}, Ai Variant | \href{https://kamalmahanna.github.io/Book-Recommendation-System/}{\underline{Book Recommender}} | \href{https://github.com/KamalMahanna/Resume-Classification-and-Parser.git}{\underline{GitHub}} \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Developed a Book Recommendation System using content-based and collaborative filtering, resulting in a 10\% boost in user retention. \item Cleaned and processed over 5,000 null values by web scrapping, improving recommendation engine performance by 30\%. \item Led the development of an Automated Resume Classification System with NLP and supervised learning, cutting manual HR effort by 40\%. \item Built robust preprocessing pipelines, enhancing resume parsing and classification accuracy by 30\%. \end{highlights} \end{onecolentry} \section{PROJECTS} \begin{onecolentry} \textbf{Audio Chat - Local AI Text \& Voice Assistant} | \href{https://github.com/KamalMahanna/Audio-Chat.git}{\underline{GitHub}} \end{onecolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Identified the growing need for private, cloud-free AI interactions and designed a local-first chat platform enabling secure text and audio conversations. \item Aimed to ensure 100\% on-device AI processing while supporting voice interaction, persistent history, and seamless deployment. \item Built a full-stack application using FastAPI, MongoDB, and Docker, integrating Ollama + LangChain for local LLMs, Whisper for speech-to-text, and Kokoro-TTS for natural voice output. \item Delivered a fully private AI assistant with persistent local chat history, instant deployment via Docker Compose, and complete cloud independence, empowering privacy-conscious users with modern AI capabilities. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{onecolentry} \textbf{AI Career Assistant Tools} | \href{https://ai-resume-creator.netlify.app/}{\underline{Resume Creator}} | \href{https://ai-career-helper.netlify.app/}{\underline{Career Helper}} \end{onecolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Built an AI-powered platform using Python, Gemini API, and Prompt Engineering, integrating resume generation, ATS scoring, and interview preparation tools to streamline job search and boost candidate success. \item Automated the creation of ATS-optimized, customizable resumes, reducing preparation time using Gemini API and enhancing relevance for recruiters. \item Added features like text summarization and personalized career guidance with NLP techniques, offering actionable insights for professional development. \item Combined multiple career tools into a cohesive web app using Flask (backend) and React (frontend), improving user experience and functionality. \end{highlights} \end{onecolentry} \vspace{0.2 cm} \begin{onecolentry} \textbf{YT-QnA – YouTube Video Question Answering App} | \href{https://yt-qna.streamlit.app/}{\underline{WebApp}} | \href{https://github.com/KamalMahanna/YT-QnA}{\underline{GitHub}} \end{onecolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item Developed an AI-powered Streamlit application that enables users to ask natural language questions about YouTube videos and receive context-aware answers. \item Implemented transcript extraction using YouTube API and Whisper (via Groq API), followed by semantic chunking and storage in ChromaDB for efficient retrieval. \item Integrated Gemini LLM to generate precise answers and audio summaries based on retrieved transcript segments, improving information accessibility from lengthy videos. \item Designed a scalable Docker-based deployment pipeline, enabling persistent vector database storage and seamless setup via containerization. \end{highlights} \end{onecolentry} \section{EDUCATION} \begin{twocolentry}{ \textit{Nov 2022} } \textbf{B.Tech in Electrical Engineering}, Biju Patnaik University of Technology \end{twocolentry} \vspace{0.10 cm} \begin{onecolentry} \begin{highlights} \item CGPA: 8.79/10 \end{highlights} \end{onecolentry} \section{CERTIFICATIONS} \begin{onecolentry} \begin{highlights} \item Data Science Certification from ExcelR \item Machine Learning Certification from \textbf{IBM} \item 5-Star Python Coder from \textbf{HackerRank} \end{highlights} \end{onecolentry} \section{ACHIEVEMENTS} \begin{onecolentry} \begin{highlights} \item Awarded "Unique and Best Data Scientist" for independently implementing null value imputation via web scraping, setting a new standard for data quality and efficiency during the Ai Variant internship. \end{highlights} \end{onecolentry} ------------------- here is the job description ------------------- Job description Roles and Responsibilities Design, develop, test, and deploy AI-powered solutions using various technologies such as AIMl, Deep Learning, and Machine Learning. Collaborate with cross-functional teams to identify business problems and implement automation solutions using tools like Blue Prism, Uipath, or Automation Anywhere. Develop data analysis skills to extract insights from large datasets and provide recommendations for process improvements. Troubleshoot issues related to deployment automation and ensure seamless integration with existing systems. Desired Candidate Profile 0-1 year of experience in Artificial Intelligence Engineering or a related field. Strong understanding of AI algorithms, machine learning algorithms, and deep learning concepts. Proficiency in programming languages such as Python Role: Automation Developer Industry Type: IT Services & Consulting Department: Engineering - Software & QA Employment Type: Full Time, Permanent Role Category: Software Development"""

In [ ]:
len(kkkk)

In [ ]:
docs[40].page_content

In [ ]:
result = chain1.invoke({'html_code':docs[0].page_content})
result.content

In [ ]:
results_gemini = [chain.invoke({"html_code": i.page_content}).content for i in docs]

In [ ]:
results_gpt2 = [chain1.invoke({"html_code": i.page_content}).content for i in docs]

In [ ]:
with open('results_gemini.txt','w+') as f:
    lines = f.writelines(results_gemini)

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['html_text','json_output'],data=[[i.page_content for i in docs],
                                                            ])


In [ ]:
import re
re.findall(r"```json\s*([\s\S]*?)```",results_gemini[1])

In [ ]:
results_gemini

In [ ]:
from test import get_response
get_response("Hello, how are you?")

In [ ]:
for i,j in enumerate(results_gemini):
    with open(f'gemini/gemini_{i}.txt', 'w') as f:
        f.write(j)
    

In [ ]:
[i.page_content for i in docs]

----

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up headless mode (optional)
chrome_options = Options()
chrome_options.add_argument("--headless=new")
# service = Service(executable_path = '/home/kamal/Desktop/Prodigy/PRODIGY_GA_01/Research/chromedriver')
# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options,
                        #   service=service
                          )

In [ ]:
driver.get("https://www.myntra.com/clothing")
source_html = driver.page_source
driver.quit()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(source_html,'html.parser')

In [ ]:
soup.find_all()

In [ ]:

try:
    # Navigate to a website
    driver.get("https://www.example.com")

    # Wait for an element to be present
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "some_element_id"))
    )

    # Extract text from the element
    data = element.text
    print(f"Extracted data: {data}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the browser
    driver.quit()

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
texts = requests.get('https://www.myntra.com/clothing').text

In [ ]:
soup = BeautifulSoup(texts,'html.parser')